In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Internal libraries
import interpolationET
import cross_correlation
import fixation_plots

In [7]:
# list to do:
#     1) Resample Labvanced to 500Hz
#     2) Resample Eyelink to 500Hz
#     3) Index suppost the same
#     4) Cross-Correlation windows - output offset in rows
#     5) offset_in_miliseconds_start = offset_in_rows * 2
#     6) offset_in_miliseconds_end = end_trigger
#     7) Calculate linear model use offset_in_miliseconds_start
# THIS IS ONLY FOR THE CROSS CORRELATION!!!
# Export all dataframes in CSV


counter = 14
offsetMedianArr = []
participantsDelayArr = []
method_2participantsDelayArr = []
# Loop over all participants
for i in range(3,counter):
    #load Labvanced
    lb = pd.read_csv('../data/lb_data/timeseries_data/p' + str(i) + '_XYTC.csv')
    #load Eyelink
    el = pd.read_csv('../data/el_data/p'+ str(i) +'.csv')
    # Showing which participants is currently ongoing
    print('running participant ' + str(i))
    # Format each dataframe from the raw export files
    lb = interpolationET.formating_labvanced(lb)
    lb = lb.set_index('time_lb')
    el = interpolationET.formating_eyelink(el)
    
# el
#     Resample Data
#     lb_resampled = cross_correlation.resampleData(lb)
    
#     #Interpolated resampled data
    df_interpolated = interpolationET.interpolation (el, lb)
    
    df_interpolated = df_interpolated.reset_index()
    
    

    delay = cross_correlation.createLagSygCorrelation(df_interpolated)

    # Convert index colum to miliseconds
    
    cross_shift = cross_correlation.compute_shift(df_interpolated)
    method_2participantsDelayArr.append(cross_shift)
    
    
    df_interpolated['ms'] = df_interpolated.index *2
    df_interpolated = df_interpolated.set_index('ms')
    
#     # Create df for first 1k ms each row represents two miliseconds
#     ms_1k = df_interpolated.loc[0:1000]
#     ms_2k = df_interpolated.loc[1000:2000]
#     ms_3k = df_interpolated.loc[2000:3000]
#     ms_4k = df_interpolated.loc[3000:4000]
#     ms_5k = df_interpolated.loc[5000:6000]
    
#     #create delay value for each participant
#     delay_ms_1k = cross_correlation.createLagSygCorrelation(ms_1k)
#     delay_ms_2k = cross_correlation.createLagSygCorrelation(ms_2k)
#     delay_ms_3k = cross_correlation.createLagSygCorrelation(ms_3k)

    participantsDelayArr.append(delay)
#     break
# #     print('Show total Lag for each participants = ' + str(participantsDelayArr))
    
#     # Each value represents number of rows, each row is spaced by 2ms
#     vector_1d = np.array(participantsDelayArr)*2
#     print('The array of whole dataset LAG for each participant in ms' + str(vector_1d))
    
    fixation_plots.timeSeriesSyncPlot(df_interpolated.loc[0:100].time_lb, df_interpolated.loc[0:100].Y_lb, df_interpolated.loc[0:100].Y_el)
    
    fixation_plots.timeSeriesSyncPlot(df_interpolated.loc[0:300].time_lb, df_interpolated.loc[0:300].Y_lb, df_interpolated.loc[0:300].Y_el)
    fixation_plots.timeSeriesSyncPlot(df_interpolated.time_lb, df_interpolated.Y_lb, df_interpolated.Y_el)
    break
#     fixation_plots.timeSeriesSyncPlot(df_interpolated.time_lb, df_interpolated.Y_lb, df_interpolated.Y_el)

running participant 3


,X_el,Y_el,Tracker_Time,time_el
t,,,,
1643104533492,719.5,464.5,557179,1643104533492
1643104533492,719.4,464.3,557181,1643104533492
1643104533492,719.4,464.1,557183,1643104533492
1643104533492,719.7,463.9,557185,1643104533492
1643104533492,719.9,463.9,557187,1643104533492
...,...,...,...,...
1643104533490,714.7,377.8,2230219,1643104533490
1643104533490,714.6,379.8,2230221,1643104533490
1643104533490,714.4,383.2,2230223,1643104533490


In [32]:
print('cross correlation array for the build in pandas method')
print(participantsDelayArr)
print('cross correlation array for the fft method')
print(method_2participantsDelayArr)

cross correlation array for the build in pandas method
[-11]
cross correlation array for the fft method
[-11]
